##### Pre-encoding all parts from the warehouse into feature vector form, for better storage and real-time access

In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import sys
sys.path.append("/home/beast/Desktop/vlassis/retrieval2/experiments")

import torch
from torch.utils.data import DataLoader, random_split
import numpy as np
from tqdm import tqdm
from diffusers.optimization import get_cosine_schedule_with_warmup

from scripts.dataset import Items3Dataset, Warehouse4Dataset, warehouse4_collate_fn
from scripts.visualization import quick_vis, quick_vis_with_parts, quick_vis_many
from scripts.visualization import plot_histogram, visualize_distribution_over_time, rotation_matrix_from_vectors2
from scripts.visualization import quick_vis_pretty, quick_vis_with_parts_pretty
from scripts.model import *
from scripts.logger import LivePlot
from scripts.metrics import AccuracyMultiClass
from scripts.utils import map_labels, generate_label_map, normalize_parts, normalize_parts_1

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


##### Choose and instantiate the model, load the appropriate checkpoint

In [2]:
categories = [0, 1, 3]

#Load the model we want
model = PartFinderPipeline2(in_channels = 3, out_channels = 384,
                           num_classes = len(categories),
                           num_attention_blocks = 3,
                           pos_emb_dim = 3,
                           pool_method = "cls_token_pool"
                          ).cuda()

#CLS_allcats_T7_partnet_pointnetnew_batchless.pt
model.load_state_dict(torch.load("/home/beast/Desktop/vlassis/retrieval2/checkpoints/CLS_allcats_T7_shapenet.pt"))
model.eval()
model = model.cuda()

In [3]:
#Encoding ALL samples, not just the specified categories. Works by design of the warehouse dataset
data_path = '/home/beast/Desktop/vlassis/retrieval2/experiments/data/vectors_warehouse_shapenet'
categories = [0, 1, 3]
warehouse = Warehouse4Dataset(cat = None, path=data_path)

Initializing warehouse dataset
Warehouse dataset initialization complete (t = 0.03170418739318848)


##### Encode every sample

In [4]:
with torch.no_grad():
    for i, (sample, label, part_label, vec) in tqdm(enumerate(warehouse)):

        #transferring to gpu
        sample = torch.from_numpy(sample).cuda().float()

        #normalizing translation
        sample = sample - sample.mean(dim=0)

        #DO NOT ROTATE, THE WAREHOUSE DATASET IS PREROTATED
        
        ####IF YOU CHANGE THE PARTS TO BE FED INTO THE MODEL HAVING NORMALIZED THEIR SCALE
        #DO THE SAME HERE
        
        #creating a dummy PID
        pid = torch.zeros(sample.shape[0]).cuda().long()

        #running the model - features: 1 x F
        features = model.forward_encoder(sample, pid)
        
        #adding the encoding to the warehouse data files (Warehouse4 only)
        warehouse.add_encoding(i, features.squeeze().cpu())

62120it [02:26, 424.78it/s]
